Imports

In [4]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
torch.cuda.is_available()

True

Download datasets

In [10]:
!python download_datasets.py -d ModelNet40
!python download_datasets.py -d 7Scenes

--2022-01-07 19:15:26--  http://download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip
Resolving download.microsoft.com (download.microsoft.com)... 2a02:26f0:3500:583::e59, 2a02:26f0:3500:595::e59, 184.30.20.109
Connecting to download.microsoft.com (download.microsoft.com)|2a02:26f0:3500:583::e59|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip [following]
--2022-01-07 19:15:26--  https://download.microsoft.com/download/2/8/5/28564B23-0828-408F-8631-23B1EFF1DAC8/tsdf.zip
Connecting to download.microsoft.com (download.microsoft.com)|2a02:26f0:3500:583::e59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 158426584 (151M) [application/octet-stream]
Saving to: ‘data/tsdf.zip’

tsdf.zip            100%[===================>] 151,09M  6,10MB/s    in 28s     

2022-01-07 19:15:55 (5,34 MB/s) - ‘data/tsdf.zip’

Run simple configuration

In [7]:
# run simple configuration

Visualize results

In [8]:
# add visualization